In [7]:
from datetime import date
import pandas as pd
import numpy as np
import holidays
from deep_translator import  GoogleTranslator
from connection import connect

In [15]:
SCHEMA='production'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

['product_list_price_history', 'product_model', 'product_model_illustration', 'product_model_prod_desc_culture', 'product_subcategory', 'scrap_reason', 'transaction_history', 'transaction_history_archive', 'bill_of_materials', 'unit_measure', 'document', 'work_order', 'culture', 'illustration', 'location', 'product', 'product_category', 'product_cost_history', 'product_photo', 'product_product_photo', 'product_review', 'product_description', 'work_order_routing', 'product_document', 'product_inventory']


In [16]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})


In [17]:
dim_fecha['day_number_of_week'] = ((dim_fecha['date'].dt.day_of_week + 1) % 7) + 1
dim_fecha['day_number_of_month']=dim_fecha['date'].dt.day
dim_fecha['day_number_of_year']=dim_fecha['date'].dt.day_of_year
dim_fecha['week_number_of_year'] = dim_fecha['date'].apply(
    lambda d: ((d - pd.Timestamp(d.year, 1, 1)).days + pd.Timestamp(d.year, 1, 1).weekday() + 1) // 7 + 1
)
dim_fecha['month_number_of_year']=dim_fecha['date'].dt.month
dim_fecha['calendar_quarter']=dim_fecha['date'].dt.quarter
dim_fecha['calendar_year']=dim_fecha['date'].dt.year
dim_fecha['calendar_semester']=(((dim_fecha['date'].dt.month-1)//6)+1)
dim_fecha['fiscal_year']=dim_fecha['date'].apply(lambda d: d.year if d.month <=7 else d.year+1)
dim_fecha['fiscal_month']=dim_fecha['date'].apply(lambda d:((d.month)-7)%12+1)
dim_fecha['fiscal_quarter']=((dim_fecha['fiscal_month']-1)//3) + 1
dim_fecha['fiscal_semester']=((dim_fecha['fiscal_month']-1)//6) + 1

dim_fecha.drop('fiscal_month',inplace=True,axis=1)

#languages


english_days = {
    1: 'Sunday',
    2: 'Monday',
    3: 'Tuesday',
    4: 'Wednesday',
    5: 'Thursday',
    6: 'Friday',
    7: 'Saturday'
}

spanish_days = {
    1: 'Domingo',
    2: 'Lunes',
    3: 'Martes',
    4: 'Miércoles',
    5: 'Jueves',
    6: 'Viernes',
    7: 'Sábado'
}

french_days = {
    1: 'Dimanche',
    2: 'Lundi',
    3: 'Mardi',
    4: 'Mercredi',
    5: 'Jeudi',
    6: 'Vendredi',
    7: 'Samedi'
}

english_months = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

spanish_months = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}

french_months = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}




dim_fecha['english_day_name_of_week'] = dim_fecha['day_number_of_week'].map(english_days)
dim_fecha['spanish_day_name_of_week']=dim_fecha['day_number_of_week'].map(spanish_days)
dim_fecha['french_day_name_of_week']=dim_fecha['day_number_of_week'].map(french_days)

dim_fecha['english_month_name']=dim_fecha['month_number_of_year'].map(english_months)
dim_fecha['spanish_month_name']=dim_fecha['month_number_of_year'].map(spanish_months)
dim_fecha['french_month_name']=dim_fecha['month_number_of_year'].map(french_months)


dim_fecha











date  day_number_of_week  day_number_of_month  day_number_of_year  \
0    2005-01-01                   7                    1                   1   
1    2005-01-02                   1                    2                   2   
2    2005-01-03                   2                    3                   3   
3    2005-01-04                   3                    4                   4   
4    2005-01-05                   4                    5                   5   
...         ...                 ...                  ...                 ...   
3647 2014-12-27                   7                   27                 361   
3648 2014-12-28                   1                   28                 362   
3649 2014-12-29                   2                   29                 363   
3650 2014-12-30                   3                   30                 364   
3651 2014-12-31                   4                   31                 365   

      week_number_of_year  month_number_of_year  calendar_quarter  \
0                       1                     1                 1   
1                       2                     1                 1   
2                       2                     1                 1   
3                       2                     1                 1   
4                       2                     1                 1   
...                   ...                   ...               ...   
3647                   52                    12                 4   
3648                   53                    12                 4   
3649                   53                    12                 4   
3650                   53                    12                 4   
3651                   53                    12                 4   

      calendar_year  calendar_semester  fiscal_year  fiscal_quarter  \
0              2005                  1         2005               3   
1              2005                  1         2005               3   
2              2005                  1         2005               3   
3              2005                  1         2005               3   
4              2005                  1         2005               3   
...             ...                ...          ...             ...   
3647           2014                  2         2015               2   
3648           2014                  2         2015               2   
3649           2014                  2         2015               2   
3650           2014                  2         2015               2   
3651           2014                  2         2015               2   

      fiscal_semester english_day_name_of_week spanish_day_name_of_week  \
0                   2                 Saturday                   Sábado   
1                   2                   Sunday                  Domingo   
2                   2                   Monday                    Lunes   
3                   2                  Tuesday                   Martes   
4                   2                Wednesday                Miércoles   
...               ...                      ...                      ...   
3647                1                 Saturday                   Sábado   
3648                1                   Sunday                  Domingo   
3649                1                   Monday                    Lunes   
3650                1                  Tuesday                   Martes   
3651                1                Wednesday                Miércoles   

     french_day_name_of_week english_month_name spanish_month_name  \
0                     Samedi            January              Enero   
1                   Dimanche            January              Enero   
2                      Lundi            January              Enero   
3                      Mardi            January              Enero   
4                   Mercredi            January              Enero   
...                      ...                ...

In [18]:
dim_fecha['date_key']=dim_fecha['date'].dt.strftime('%Y%m%d').astype(int)


dim_fecha.rename(columns={'date':'full_date_alternate_key'},inplace=True)


dim_fecha

full_date_alternate_key  day_number_of_week  day_number_of_month  \
0                 2005-01-01                   7                    1   
1                 2005-01-02                   1                    2   
2                 2005-01-03                   2                    3   
3                 2005-01-04                   3                    4   
4                 2005-01-05                   4                    5   
...                      ...                 ...                  ...   
3647              2014-12-27                   7                   27   
3648              2014-12-28                   1                   28   
3649              2014-12-29                   2                   29   
3650              2014-12-30                   3                   30   
3651              2014-12-31                   4                   31   

      day_number_of_year  week_number_of_year  month_number_of_year  \
0                      1                    1                     1   
1                      2                    2                     1   
2                      3                    2                     1   
3                      4                    2                     1   
4                      5                    2                     1   
...                  ...                  ...                   ...   
3647                 361                   52                    12   
3648                 362                   53                    12   
3649                 363                   53                    12   
3650                 364                   53                    12   
3651                 365                   53                    12   

      calendar_quarter  calendar_year  calendar_semester  fiscal_year  \
0                    1           2005                  1         2005   
1                    1           2005                  1         2005   
2                    1           2005                  1         2005   
3                    1           2005                  1         2005   
4                    1           2005                  1         2005   
...                ...            ...                ...          ...   
3647                 4           2014                  2         2015   
3648                 4           2014                  2         2015   
3649                 4           2014                  2         2015   
3650                 4           2014                  2         2015   
3651                 4           2014                  2         2015   

      fiscal_quarter  fiscal_semester english_day_name_of_week  \
0                  3                2                 Saturday   
1                  3                2                   Sunday   
2                  3                2                   Monday   
3                  3                2                  Tuesday   
4                  3                2                Wednesday   
...              ...              ...                      ...   
3647               2                1                 Saturday   
3648               2                1                   Sunday   
3649               2                1                   Monday   
3650               2                1                  Tuesday   
3651               2                1                Wednesday   

     spanish_day_name_of_week french_day_name_of_week english_month_name  \
0                      Sábado                  Samedi            January   
1                     Domingo                Dimanche            January   
2                       Lunes                   Lundi            January   
3                      Martes                   Mardi            January   
4                   Miércoles                Mercredi            January   
...                       ...                     ...                ...   
3647                   Sábado                  Samedi           December 

## Load

In [20]:
dim_fecha.to_sql('dim_date',etl_conn,if_exists='append',index=False)

652